# Ghost Sequences From GeoLife Trajectory 1.3 sample data

*Part of the COVID19risk project*  
*http://covid19risk.com/*  
*2020-03-04*  

*Copyright (C) 2020 Mikhail Voloshin, Mighty Data Inc.*  
*All rights reserved.*  

The objective of this Notebook is to convert the sample GeoLife trajectory data from Microsoft (https://www.microsoft.com/en-us/download/details.aspx?id=52367) into a form that can be marshalled to a browser for purposes of being rendered client-side as a heatmap that can be navigated across space and time.

The Geolife data is hundreds of megabytes ZIPped, and over 1.5 GB expanded into CSV-based "trajectory" files. This is far too big and too detailed to be usable by a client, and needs to be simplified and summarized in order to be usable.

The Geolife data is also very sparse. It's very rare for two Geolife users to be anywhere near one another at the same time, or even exhibiting activity within the same small number of days as one another.

The Geolife data also has relatively few users. There are only 182 people represented in the entire dataset.

This Notebook seeks to solve all of the above problems by imagining that each continuous record from each of these 182 people is a "ghost" that loops through the same series of actions continuously. This creates the impression of thousands of simultaneous points of continuous activity, while still permitting us to say that we're using real GPS data.


## Define parameters

In [1]:
# Define parameters

# The distance, in km, that a user has to travel before their GPS position is updated in the 
# summarized record. This is just a handy way to prevent the record from getting too big;
# it basically says that it only says that a user has moved after they've traveled more
# than GRID_SPATIAL_KM distance (or thereabouts).
# The bigger this number is, the smaller the resulting output file.
GRID_SPATIAL_KM = 2

# Path to the unzipped Geolife data folder.
DATA_DIR = "../../../../data/proof-of-concept/Geolife Trajectories 1.3"

## Search for trajectory files

In [2]:
import os

if not os.path.isdir(DATA_DIR):
    raise ValueError(DATA_DIR + " doesn't appear to be a directory that exists on this filesystem.")
if not os.path.isdir(DATA_DIR+'/Data'):
    raise ValueError(DATA_DIR + " doesn't appear to be the unzipped Geolife dataset. It doesn't contain a /Data subdirectory.")

# A trajectory folder path is any subfolder of the Data directory that has a Trajectory subfolder.
trajectory_folder_paths = [f.path for f in os.scandir(DATA_DIR+'/Data') if f.is_dir() and os.path.isdir(f.path+'/Trajectory')]

print(f'Success. {len(trajectory_folder_paths)} trajectories found.')

Success. 182 trajectories found.


## Precompute some values that will be useful to us during the computation

Spatial grid degrees were computed with the help of https://andrew.hedges.name/experiments/haversine/. Future versions of this script should use trigonometry to discretize by traversing a longitudinal line in intervals of *GRID_SPATIAL_KM* km steps from the equator, and then using the Earth's cross-section at that latitude to walk by *GRID_SPATIAL_KM* km steps from the prime meridian. It's not hard, but a little too tedious to go through the trouble of writing said function for this proof-of-concept stage.

In [3]:
CELLSIZE_DEGREES_LAT = GRID_SPATIAL_KM * 0.00899
CELLSIZE_DEGREES_LONG = GRID_SPATIAL_KM * 0.01174
CELLSIZE_DEGREES_PRECISION = 5


## Define some helper functions

In [4]:
import time
import datetime

MICROSOFT_EPOCH_START = time.mktime(datetime.datetime.strptime('1899-12-30', '%Y-%m-%d').timetuple())

def convert_microsoft_epoch_to_unixtime(dayscount):
    # Microsoft Research encoded this dataset to include a field
    # that contains the number of days that have elapsed since
    # Dec 30 (not 31), 1899. No word on whether this is midnight
    # at the *beginning* or *end* of said date, but we can validate
    # against the other date and time columns to make sure we
    # got it right (which we did).

    secondscount = dayscount * 24 * 60 * 60
    retval = MICROSOFT_EPOCH_START + secondscount
    return int(retval)
    


In [5]:
# We'll keep the last lat/long position memoed, and determine when the current position
# has wandered away from the memo by a distance of about GRID_SPATIAL_KM.
# This is probably an abuse of the dataframe apply method, but as long as it runs in
# order, we'll be okay.
# Note that we're computing movement by Manhattan metric. In the real world, we'd use
# a Euclidean metric, but this is good enough.

def check_wander(df):
    lastpos = [0, 0]
    def did_wander(row):
        if abs(row.latitude - lastpos[0]) > CELLSIZE_DEGREES_LAT or abs(row.longitude - lastpos[1]) > CELLSIZE_DEGREES_LONG:
            lastpos[0] = row.latitude
            lastpos[1] = row.longitude
            return True
        return False
        
    return df.apply(did_wander, axis=1)


## Populate our data structures
This script should take about 20 minutes to run on a 2.8 Ghz Lenovo laptop with 16 GB RAM.

In [6]:
%%time
print(f'Started at {datetime.datetime.now()}')

import math
import json
import pandas as pd

# The column meanings come from the User Guide PDF that comes with the Geolife dataset.
GEOLIFE_COLUMNS = [
    'latitude',
    'longitude',
    'reserved0',
    'altitude',
    'daysSinceMicrosoftEpoch',
    'date',
    'time'    
]

trajectories_by_ghostid = {}


for traj_path in trajectory_folder_paths:
    traj_id = traj_path.split('/')[-1]
    print(f'Processing Trajectory ID: {traj_id}')

    cells_hit_by_this_traj = set()
    
    traj_filenames = [f.path for f in os.scandir(traj_path+'/Trajectory') if f.is_file() and f.path.endswith('.plt')]
    print(f'{len(traj_filenames)} trajectory plots found.')
    
    for traj_subnum, traj_filename in enumerate(traj_filenames):
        ghost_id = f'{traj_id}-{traj_subnum:05}'
        print(f'    {ghost_id}')

        df = pd.read_csv(traj_filename, skiprows=6, names=GEOLIFE_COLUMNS)

        df['unixtime'] = df['daysSinceMicrosoftEpoch'].apply(convert_microsoft_epoch_to_unixtime)
        
        df['wander'] = check_wander(df)
        
        wanderdf = df.loc[df['wander'], ['latitude', 'longitude', 'unixtime']]
        wanderdf['seconds'] = (wanderdf['unixtime'] - wanderdf['unixtime'].shift()).fillna(0).astype('int64')
        

        wanderdf['lat_fixedpt'] = (wanderdf['latitude'] * math.pow(10, CELLSIZE_DEGREES_PRECISION)).astype('int64')
        wanderdf['long_fixedpt'] = (wanderdf['longitude'] * math.pow(10, CELLSIZE_DEGREES_PRECISION)).astype('int64')
        
        trimdf = wanderdf.loc[:, ['lat_fixedpt','long_fixedpt', 'seconds']]
        
        ghost_traj = json.loads(trimdf.to_json(orient='values'))
        trajectories_by_ghostid[ghost_id] = ghost_traj
            
    print(f'\nTrajectory ID {traj_id} done.\n')


Started at 2020-03-16 00:50:13.003129
Processing Trajectory ID: 000
171 trajectory plots found.
    000-00000
    000-00001
    000-00002
    000-00003
    000-00004
    000-00005
    000-00006
    000-00007
    000-00008
    000-00009
    000-00010
    000-00011
    000-00012
    000-00013
    000-00014
    000-00015
    000-00016
    000-00017
    000-00018
    000-00019
    000-00020
    000-00021
    000-00022
    000-00023
    000-00024
    000-00025
    000-00026
    000-00027
    000-00028
    000-00029
    000-00030
    000-00031
    000-00032
    000-00033
    000-00034
    000-00035
    000-00036
    000-00037
    000-00038
    000-00039
    000-00040
    000-00041
    000-00042
    000-00043
    000-00044
    000-00045
    000-00046
    000-00047
    000-00048
    000-00049
    000-00050
    000-00051
    000-00052
    000-00053
    000-00054
    000-00055
    000-00056
    000-00057
    000-00058
    000-00059
    000-00060
    000-00061
    000-00062
    000-00063
    000-

    003-00144
    003-00145
    003-00146
    003-00147
    003-00148
    003-00149
    003-00150
    003-00151
    003-00152
    003-00153
    003-00154
    003-00155
    003-00156
    003-00157
    003-00158
    003-00159
    003-00160
    003-00161
    003-00162
    003-00163
    003-00164
    003-00165
    003-00166
    003-00167
    003-00168
    003-00169
    003-00170
    003-00171
    003-00172
    003-00173
    003-00174
    003-00175
    003-00176
    003-00177
    003-00178
    003-00179
    003-00180
    003-00181
    003-00182
    003-00183
    003-00184
    003-00185
    003-00186
    003-00187
    003-00188
    003-00189
    003-00190
    003-00191
    003-00192
    003-00193
    003-00194
    003-00195
    003-00196
    003-00197
    003-00198
    003-00199
    003-00200
    003-00201
    003-00202
    003-00203
    003-00204
    003-00205
    003-00206
    003-00207
    003-00208
    003-00209
    003-00210
    003-00211
    003-00212
    003-00213
    003-00214
    00

    005-00001
    005-00002
    005-00003
    005-00004
    005-00005
    005-00006
    005-00007
    005-00008
    005-00009
    005-00010
    005-00011
    005-00012
    005-00013
    005-00014
    005-00015
    005-00016
    005-00017
    005-00018
    005-00019
    005-00020
    005-00021
    005-00022
    005-00023
    005-00024
    005-00025
    005-00026
    005-00027
    005-00028
    005-00029
    005-00030
    005-00031
    005-00032
    005-00033
    005-00034
    005-00035
    005-00036
    005-00037
    005-00038
    005-00039
    005-00040
    005-00041
    005-00042
    005-00043
    005-00044
    005-00045
    005-00046
    005-00047
    005-00048
    005-00049
    005-00050
    005-00051
    005-00052
    005-00053
    005-00054
    005-00055
    005-00056
    005-00057
    005-00058
    005-00059
    005-00060
    005-00061
    005-00062
    005-00063
    005-00064
    005-00065
    005-00066
    005-00067
    005-00068
    005-00069
    005-00070
    005-00071
    00

    011-00141
    011-00142
    011-00143
    011-00144
    011-00145
    011-00146
    011-00147
    011-00148
    011-00149
    011-00150
    011-00151
    011-00152
    011-00153
    011-00154
    011-00155
    011-00156
    011-00157
    011-00158
    011-00159
    011-00160
    011-00161
    011-00162
    011-00163
    011-00164
    011-00165
    011-00166
    011-00167
    011-00168
    011-00169
    011-00170
    011-00171
    011-00172
    011-00173
    011-00174
    011-00175
    011-00176
    011-00177
    011-00178
    011-00179
    011-00180
    011-00181
    011-00182
    011-00183
    011-00184
    011-00185
    011-00186
    011-00187
    011-00188
    011-00189
    011-00190
    011-00191
    011-00192
    011-00193
    011-00194
    011-00195
    011-00196
    011-00197
    011-00198
    011-00199
    011-00200

Trajectory ID 011 done.

Processing Trajectory ID: 012
77 trajectory plots found.
    012-00000
    012-00001
    012-00002
    012-00003
    012-00004
    012

    015-00007
    015-00008
    015-00009
    015-00010
    015-00011
    015-00012
    015-00013
    015-00014
    015-00015
    015-00016
    015-00017
    015-00018
    015-00019
    015-00020
    015-00021
    015-00022
    015-00023
    015-00024
    015-00025
    015-00026
    015-00027
    015-00028
    015-00029
    015-00030
    015-00031
    015-00032
    015-00033
    015-00034
    015-00035
    015-00036
    015-00037
    015-00038
    015-00039
    015-00040
    015-00041
    015-00042
    015-00043
    015-00044
    015-00045
    015-00046
    015-00047
    015-00048
    015-00049
    015-00050
    015-00051
    015-00052
    015-00053
    015-00054
    015-00055
    015-00056
    015-00057
    015-00058
    015-00059
    015-00060
    015-00061
    015-00062
    015-00063
    015-00064
    015-00065
    015-00066

Trajectory ID 015 done.

Processing Trajectory ID: 016
51 trajectory plots found.
    016-00000
    016-00001
    016-00002
    016-00003
    016-00004
    016

...

    153-01266
    153-01267
    153-01268
    153-01269
    153-01270
    153-01271
    153-01272
    153-01273
    153-01274
    153-01275
    153-01276
    153-01277
    153-01278
    153-01279
    153-01280
    153-01281
    153-01282
    153-01283
    153-01284
    153-01285
    153-01286
    153-01287
    153-01288
    153-01289
    153-01290
    153-01291
    153-01292
    153-01293
    153-01294
    153-01295
    153-01296
    153-01297
    153-01298
    153-01299
    153-01300
    153-01301
    153-01302
    153-01303
    153-01304
    153-01305
    153-01306
    153-01307
    153-01308
    153-01309
    153-01310
    153-01311
    153-01312
    153-01313
    153-01314
    153-01315
    153-01316
    153-01317
    153-01318
    153-01319
    153-01320
    153-01321
    153-01322
    153-01323
    153-01324
    153-01325
    153-01326
    153-01327
    153-01328
    153-01329
    153-01330
    153-01331
    153-01332
    153-01333
    153-01334
    153-01335
    153-01336
    15

    153-01858
    153-01859
    153-01860
    153-01861
    153-01862
    153-01863
    153-01864
    153-01865
    153-01866
    153-01867
    153-01868
    153-01869
    153-01870
    153-01871
    153-01872
    153-01873
    153-01874
    153-01875
    153-01876
    153-01877
    153-01878
    153-01879
    153-01880
    153-01881
    153-01882
    153-01883
    153-01884
    153-01885
    153-01886
    153-01887
    153-01888
    153-01889
    153-01890
    153-01891
    153-01892
    153-01893
    153-01894
    153-01895
    153-01896
    153-01897
    153-01898
    153-01899
    153-01900
    153-01901
    153-01902
    153-01903
    153-01904
    153-01905
    153-01906
    153-01907
    153-01908
    153-01909
    153-01910
    153-01911
    153-01912
    153-01913
    153-01914
    153-01915
    153-01916
    153-01917
    153-01918
    153-01919
    153-01920
    153-01921
    153-01922
    153-01923
    153-01924
    153-01925
    153-01926
    153-01927
    153-01928
    15

    163-00215
    163-00216
    163-00217
    163-00218
    163-00219
    163-00220
    163-00221
    163-00222
    163-00223
    163-00224
    163-00225
    163-00226
    163-00227
    163-00228
    163-00229
    163-00230
    163-00231
    163-00232
    163-00233
    163-00234
    163-00235
    163-00236
    163-00237
    163-00238
    163-00239
    163-00240
    163-00241
    163-00242
    163-00243
    163-00244
    163-00245
    163-00246
    163-00247
    163-00248
    163-00249
    163-00250
    163-00251
    163-00252
    163-00253
    163-00254
    163-00255
    163-00256
    163-00257
    163-00258
    163-00259
    163-00260
    163-00261
    163-00262
    163-00263
    163-00264
    163-00265
    163-00266
    163-00267
    163-00268
    163-00269
    163-00270
    163-00271
    163-00272
    163-00273
    163-00274
    163-00275
    163-00276
    163-00277
    163-00278
    163-00279
    163-00280
    163-00281
    163-00282
    163-00283
    163-00284
    163-00285
    16

    163-00805
    163-00806
    163-00807
    163-00808

Trajectory ID 163 done.

Processing Trajectory ID: 164
7 trajectory plots found.
    164-00000
    164-00001
    164-00002
    164-00003
    164-00004
    164-00005
    164-00006

Trajectory ID 164 done.

Processing Trajectory ID: 165
26 trajectory plots found.
    165-00000
    165-00001
    165-00002
    165-00003
    165-00004
    165-00005
    165-00006
    165-00007
    165-00008
    165-00009
    165-00010
    165-00011
    165-00012
    165-00013
    165-00014
    165-00015
    165-00016
    165-00017
    165-00018
    165-00019
    165-00020
    165-00021
    165-00022
    165-00023
    165-00024
    165-00025

Trajectory ID 165 done.

Processing Trajectory ID: 166
8 trajectory plots found.
    166-00000
    166-00001
    166-00002
    166-00003
    166-00004
    166-00005
    166-00006
    166-00007

Trajectory ID 166 done.

Processing Trajectory ID: 167
385 trajectory plots found.
    167-00000
    167-00001
    167-000

    169-00026
    169-00027
    169-00028
    169-00029
    169-00030
    169-00031
    169-00032
    169-00033
    169-00034
    169-00035

Trajectory ID 169 done.

Processing Trajectory ID: 170
5 trajectory plots found.
    170-00000
    170-00001
    170-00002
    170-00003
    170-00004

Trajectory ID 170 done.

Processing Trajectory ID: 171
5 trajectory plots found.
    171-00000
    171-00001
    171-00002
    171-00003
    171-00004

Trajectory ID 171 done.

Processing Trajectory ID: 172
21 trajectory plots found.
    172-00000
    172-00001
    172-00002
    172-00003
    172-00004
    172-00005
    172-00006
    172-00007
    172-00008
    172-00009
    172-00010
    172-00011
    172-00012
    172-00013
    172-00014
    172-00015
    172-00016
    172-00017
    172-00018
    172-00019
    172-00020

Trajectory ID 172 done.

Processing Trajectory ID: 173
6 trajectory plots found.
    173-00000
    173-00001
    173-00002
    173-00003
    173-00004
    173-00005

Trajectory I

## Examine our results.
Take a look at how much data we've produced

In [7]:
print(f'Number of ghosts: {len(trajectories_by_ghostid)}')

Number of ghosts: 18670


In [8]:
ghostlens = [len(traj) for traj in trajectories_by_ghostid.values()]

In [9]:
print(f'Average number of entries per ghost: {sum(ghostlens)/len(ghostlens)}')

Average number of entries per ghost: 9.562131762185324


## Save output
Here's what we've been waiting for! Save our output, along with the parameters we used for creating it.

In [10]:
results = {}

# Record the grid parameters.
results['gridparams'] = {
    'spatial-cell-size-km': GRID_SPATIAL_KM,
    'fixed-point-precision': CELLSIZE_DEGREES_PRECISION,
}

# And of course, the trajectories!
results['trajectories'] = trajectories_by_ghostid

In [11]:

with open('trajectories_ghosts.json', 'w') as f:
    json.dump(results, f)